In [4]:
# We want to use the array job feature of the cluster to run multiple experiments in parallel
# in the job file we'd like the array index to influence the configuration of the experiment


job_template = """#!/bin/bash

#SBATCH --partition=staging
#SBATCH --gpus=0
#SBATCH --job-name={job_name}
#SBATCH --ntasks={n_tasks}
#SBATCH --cpus-per-task=1
#SBATCH --time=2:00:00
#SBATCH --mem=7000M
#SBATCH --output=../outputs/{job_name}_%A_%a.out

module purge
module load 2022
module load Anaconda3/2022.05

# Activate your environment
source activate EVOMAN
# Change to the directory where your code is located
cd ../evoman_framework
# Run your code
srun -n {n_tasks} python -u run_optim.py {config_str} --enemy_sets $SLURM_ARRAY_TASK_ID --start_new --run_evolution --n_runs 1 --name {job_name}"""

n_tasks = 30

# Generate job files like the above for each of the following configurations
# and save them in ../jobs/{job_name}.job
# The job files will be used to run the experiments on the cluster

"""
    parser = argparse.ArgumentParser()
    parser.add_argument('--randomini',              type=str, default='no',         help='usage: --randomini yes/no')
    parser.add_argument('--multi_ini',              type=bool, default=False,       help='usage: --multi_ini True/False')
    parser.add_argument('--normalization_method',   type=str, default='default',    help='usage: --normalization_method default/domain_specific/around_0')
    parser.add_argument('--fitness_method',         type=str, default='rank',    help='usage: --fitness_method default/balanced/rank')
    parser.add_argument('--pick_parent_method',     type=str, default='multinomial', help='usage: --pick_parent_method multinomial/tournament/greedy')
    parser.add_argument('--survivor_method',        type=str, default='multinomial', help='usage: --survivor_method greedy/multinomial/tournament')
    parser.add_argument('--crossover_method',       type=str, default='none',       help='usage: --crossover_method none/default/ensemble')
    parser.add_argument('--mutation_type',          type=str, default='normal',     help='usage: --mutation_type normal/stochastic_decaying')
    parser.add_argument('--enemy_sets',             type=str, default='1,2,3,4,5,6,7,8',   help='usage: --enemy_sets 1,2,3,4,5,6,7,8/12,13/1')
"""

argument_dict = {
    "randomini": ["yes", "no"],
    "multi_ini": [True, False],
    "normalization_method": ["default", "domain_specific", "around_0"],
    "fitness_method": ["default", "balanced", "rank"],
    "pick_parent_method": ["tournament", "greedy", "multinomial"],
    "survivor_method": ["multinomial", "greedy", "tournament"],
    "crossover_method": ["none", "default", "ensemble"],
    "mutation_type": ["normal", "stochastic_decaying"],
}
arg_defaults = dict(
    randomini="no",
    multi_ini=False,
    normalization_method="default",
    fitness_method="rank",
    pick_parent_method="multinomial",
    survivor_method="multinomial",
    crossover_method="none",
    mutation_type="normal",
)

def dos2unix(filepath):
    content = ''
    outsize = 0
    with open(filepath, 'rb') as infile:
        content = infile.read()
    with open(filepath, 'wb') as output:
        for line in content.splitlines():
            outsize += len(line) + 1
            output.write(line + b'\n')

def save_job(name, job):
    with open(f"../jobs/{name}.job", "w", encoding="utf-8") as f:
        f.write(job)

    # Use dos2unix to convert the file to UNIX line endings
    dos2unix(f"../jobs/{name}.job")

# Make one job for default configuration
config_str = ""
job_name = "default"
job = job_template.format(job_name=job_name, config_str=config_str, n_tasks=n_tasks)
save_job(job_name, job)

for arg, values in argument_dict.items():
    for value in values:
        if value == arg_defaults[arg]:
            continue
        config_str = f"--{arg} {value}"
        job_name = f"{arg}_{value}"
        job = job_template.format(job_name=job_name, config_str=config_str, n_tasks=n_tasks)
        save_job(job_name, job)